# Chapter 05 트리 알고리즘

### 05-1 결정 트리

알코올, 도수, 당도, pH값으로 와인 분류하기. ( Red or White )

로지스틱 회귀로 분류하기:

In [ ]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.head()

# class: 0 --> 레드 와인
# class: 1 --> 화이트 와인

In [ ]:
wine.info()  # info(): 데이터프레임 각 열의 데이터 타입과 누락된 데이터가 있는지 확인

In [ ]:
wine.describe()  # describe(): 열에 대한 간략한 통계 출력 (최소, 최대, 평균, 표준편차 등...)

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()  # 각 특성 데이터
target = wine['class'].to_numpy()   # 타겟값

In [ ]:
# 훈련세트, 테스트세트 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
print(train_input.shape, test_input.shape)

In [ ]:
# 특성 스케일링 (정규화)
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [ ]:
# 로지스틱 회귀 모델 훈련
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_scaled, train_target)
print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

In [ ]:
print(lr.coef_, lr.intercept_)

각 coef에 특성을 곱해서 더하고 intercept를 더하면 z값이 됨

그걸 시그모이드함수에 넣고 0보다 크면 양성(화이트와인), 작으면 음성(레드와인) 이다.

결정 트리

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(train_scaled, train_target)
print(dt.score(train_scaled, train_target))
print(dt.score(test_scaled, test_target))

In [ ]:
# 결정트리 모델 그리기
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
plt.figure(figsize=(10,7))
plot_tree(dt)
plt.show()

In [ ]:
# 위 그림이 복잡하니 확대해서 그리기
plt.figure(figsize=(10,7))
plot_tree(dt, max_depth=1, filled=True, feature_names=['alcohol', 'sugar', 'pH'])
# filled=True: 색 채우기 --> 어떤 클래스의 비율이 높아지면 점점 진한 색으로 표시
plt.show()

왼쪽이 Yes, 오른쪽이 No

- 테스트 조건 (sugar)

- 불순도 (gini)

- 총 샘플 수 (samples)

- 클래스별 샘플 수 (value) : (음성클래스 수, 양성클래스 수)

결정트리에서 예측하는 방법 : 리프노드에서 가장 많은 클래스가 예측 클래스가 된다

**gini ( 지니 불순도 )** : 테스트 조건을 결정

지니 불순도 = 1 - ( 음성클래스 비율²+ 양성클래스 비율² )

- 불순도 == 0인 노드 : 순수 노드

결정트리 모델은 부모노드와 자식녿의 불순도 차이가 가능한 크도록 트리를 성장시킨다!

--> 불순도 차이:

부모의 불순도 - (왼쪽노드 샘플수 / 부모의 샘플수) x 왼쪽노드 불순도 - (오른쪽노드 샘플수 / 부모의 샘플수) x 오른쪽노드 샘플수

불순도 차이를 **정보이득** 이라고 한다.

즉, 결정트리는 정보이득이 최대가 되도록 데이터를 나눈다, 노드를 순수하게 나눌수록 정보이득이 최대가 된다.

새로운 샘플을 예측할 때에는 노드의 질문에 따라 트리를 이동하여 마지막에 도달한 노드의 클래스 비율을 보고 예측한다!

In [ ]:
# 가지치기 : 결정트리가 끝까지 성장하는 것을 제한
dt = DecisionTreeClassifier(max_depth=3, random_state=42) # max_depth=3 으로 제한
dt.fit(train_scaled, train_target)
print(dt.score(train_scaled, train_target))
print(dt.score(test_scaled, test_target))

In [ ]:
plt.figure(figsize=(20,15))
plot_tree(dt, filled=True, feature_names=['alcohol', 'sugar', 'pH'])
plt.show()

3번째 노드만 음성클래스가 더 많다 --> 이 노드에 도착해야만 레드와인으로 예측한다.

In [ ]:
dt = DecisionTreeClassifier(max_depth=3, random_state=42)
dt.fit(train_input, train_target)
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

# 특성값의 스케일은 결정트리 알고리즘에 아무런 영향을 미치지 않는다
# --> 표준화 전처리를 할 필요가 없다!..
# 결과 같음

In [ ]:
plt.figure(figsize=(20,15))
plot_tree(dt, filled=True, feature_names=['alcohol', 'sugar', 'pH'])
plt.show()

In [ ]:
print(dt.feature_importances_)  # 특성 중요도 출력
# 루트노드와 depth=1 에서 sugar을 사용했기 때문에 sugar의 중요도가 제일 높을것으로 예상
# --> 결정트리의 특성 중요도를 특성 선택에 활용할 수 있다!